# The Wealth of Cities
## Predicting the Wealth of a City from Sattelite Imagery

Accurate measurements of the economic characteristics of populations critically influence research and policy. Such measurements can shape decicisions by governments and how to allocate resources and provide infrastructure to improve human livelihoods in a wide-range of situations. Although economic data is readily available for some developing nations, many regions of the modern world remain unexposed to the benefits of economic analysis where regions lack key measures of economic development and efficiency. Regions such as parts of Africa conduct little to no economic surveys or other means of collecting data on their financial situations. We attempt to address this problem by using publicly available satellite imagery to predict the wealth of a city (or, more generally, a geographic region) based on fundamental features identified in these images and running them through a convolutional neural network. Not only would this method be applicable to regions that lack economic data, but could also be applied to cities with a wealth of economic information on a macro level but a dearth on a micro level. For example, cities in America, despite having lots of economic data on state and county levels, could benefit from understanding more granular information in order to improve policy decisions for infrastructure and public support. 

In order for this approach to work, we need to be able to extract relevant features from the images in order to train our machine learning model. Our model will not be able to predict the wealth of individual houses (i.e., families), but will work on clusters of houses (i.e., neighborhoods) because of the complexity of wealth measurements and tendency for neighborhood to be at a nearly homogenous economic level. As a result, we will need to extract "cluster" features to process with our (NEURAL NETWORK).

Thinking about the kinds of features that would elucidate the wealth of a region, we can start to identify what we need to extract (in some way) from the images. One of the first and most common thoughts is to get satellite imagery of the region at night and observe the nighlight intensity; more lights at night tend to correspond with more wealth while less lights at night tend to correspond with poorer areas. Our group has also thought of the following ideas as means to identify wealth:
- Number of cars
- Percentage of greenspace
- Number of high-rises
- What time traffic occurs at
- Housing density
- Aerospace/nautical infrastructure

The number of of cars tends to be a good indicator or whether a city has passed a certain threshold for wealth. Yes, some cities that are poorer than others will have more cars, but cities that have no cars tend to be the poorest, so we can figure out a baseline level for the wealth of a city if we can extract the number of cars from the image.

Percentage of greenspace is perhaps even less reliable than the number of cars, but can also establish relative rankings of wealth between multiple cities. Cities with lots of public funds, and consequently wealth, will tend to spend money on maintaining public greenspaces. Granted, some rural regions tend to also have a lot of greenspace in the form of farms or undeveloped land, so in this case greenspace does not correspond to higher wealth. However, if we can ensure that the imagery we are looking at represents a urban city, we could perhaps take into  greenspace into account to predict the level of wealth.

Number of high-rises is definitely a critical feature of a city's wealth. However, extracting this information from satellite imagery proves to be tricky because of the flattness of the images. One way to get around this is to analyze the shadows produced by buildings at different times of the day. If the buildings are tall, they will cast long shadows at all times of the day (not only briefly in the morning and night).

Housing density is highly correlated to the "urban-ness" of a region, which in turn is suggestive of the wealth of a city. Rural areas (i.e., poorer, generally) have a lower housing density while urban areas (i.e., wealthier, generally) have a higher housing density. Granted, there are exceptions to this trend, but generally this fact will hold and is one of the easier features to extract from satellite imagery.

We will be getting our images from Planet.org, a publically available database of satellite imagery from the last few years that covers most of the world. Unfortunately, API access is limited to California so we will only be able to run our model using data from California, but there is no reason that this method would not work given more input data from around the world.

In this notebook, we'll take you through the entire process from setting up the program to download images and extract features to running the data through the machine learning pipeline and getting a predicted wealth score for input data. 

First, we'll input the necessary modules. `json` and `io` are just used to load in our Planet.org API key. You can sign up for a free account at https://www.planet.com/. The approval process will take a few days, but after receiving your API key, this entire notebook can be completed in one sitting. We will be using the `requests` module to make API requests for the satellite imagery, which requires authorization using the `requests.auth` module.

In [19]:
import json, io
import requests
from requests.auth import HTTPBasicAuth
from PIL import Image

In [8]:
# LOADS in your PLANET_API_KEY from the config_secret.json file
with io.open("config_secret.json") as cred:
    API_KEY = json.load(cred)["API_KEY"]

In [73]:
def open_image(content):
    return Image.open(io.BytesIO(content)).convert("RGBA")

BASE_URL = "https://maps.googleapis.com/maps/api/staticmap"
BASE_PAYLOAD = [("size", "600x600"), ("key", API_KEY)]
STYLE_PAYLOAD = [("style", "feature:road|element:geometry|color:0x00ff00"),
                 ("style", "feature:landscape.man_made|element:geometry.fill|color:0x000000"),
                 ("style", "element:labels|visibility:off"),
                 ("style", "feature:poi|element:geometry|color:0xff0000"), # poi.park remove
                 ("style", "feature:water|element:geometry|color:0x0000ff"),
                 ("style", "feature:landscape.natural|element:geometry.fill|color:0xff00ff")]
SATELLITE_PAYLOAD = BASE_PAYLOAD + [("maptype", "satellite")]
ROADMAP_PAYLOAD = BASE_PAYLOAD + STYLE_PAYLOAD + [("maptype", "roadmap")]

payload =  ROADMAP_PAYLOAD + [("center", "40.714728,-73.998672"), ("zoom", "9")]
r = requests.get(BASE_URL, params=payload)
im = open_image(r.content)
im.show()

To get an idea of what these satellite images look like, we will show you how to download a single image and then proceed to, what Planet calls, an Area of Interest, or AOI. First, we define a geometry, which is a collection of latitude and longtitude points  that forms a polygon around the area you would like to get pictures from. Remember that Planet API only works with California right now, so if you want to change the coordinates, make sure they remain within the state. Our example geometry is centered on a reservoir in Redding, CA. Next, we'll need to define filters for the Planet API; these include the geometry filter discussed above, as well as date range filters (only getting images within a specified date range), cloud cover filters (perhaps you only want to look at images on clear day), and many more. We then send this request to the Stats API endpoint to see how many possible images there are that fit our criteria. In our example, there are 30 images taken of Redding, CA within the date range that have less than 50% cloud cover.